In [1]:
spark

In [2]:
import pandas as pd

In [3]:
bucket = 'my-bigdata-project-ns-fhvhv'
file_list = ['fhvhv_tripdata_2020-01.parquet',
             'fhvhv_tripdata_2020-02.parquet',
             'fhvhv_tripdata_2020-03.parquet',
             'fhvhv_tripdata_2020-04.parquet',
             'fhvhv_tripdata_2020-05.parquet',
             'fhvhv_tripdata_2020-06.parquet',
             'fhvhv_tripdata_2020-07.parquet',
             'fhvhv_tripdata_2020-08.parquet',
             'fhvhv_tripdata_2020-09.parquet',
             'fhvhv_tripdata_2020-10.parquet',
             'fhvhv_tripdata_2020-11.parquet',
             'fhvhv_tripdata_2020-12.parquet',
             'fhvhv_tripdata_2021-01.parquet',
             'fhvhv_tripdata_2021-02.parquet',
             'fhvhv_tripdata_2021-03.parquet',
             'fhvhv_tripdata_2021-04.parquet',
             'fhvhv_tripdata_2021-05.parquet',
             'fhvhv_tripdata_2021-06.parquet',
             'fhvhv_tripdata_2021-07.parquet',
             'fhvhv_tripdata_2021-08.parquet',
             'fhvhv_tripdata_2021-09.parquet',
             'fhvhv_tripdata_2021-10.parquet',
             'fhvhv_tripdata_2021-11.parquet',
             'fhvhv_tripdata_2021-12.parquet',]
base_directory = f"gs://my-bigdata-project-ns-fhvhv/Landing/"

for file_name in file_list:
    file_path = base_directory + file_name
    
    df = pd.read_parquet(file_path)

In [4]:
# Clean the data
# Remove records with missing data
df_cleaned = df.dropna()

# Fill in missing values where appropriate
# For example, you can fill missing values in numeric columns with the mean
numeric_columns = df_cleaned.select_dtypes(include=['float64', 'int64']).columns
df_cleaned[numeric_columns] = df_cleaned[numeric_columns].fillna(df_cleaned[numeric_columns].mean())

# Drop unneeded columns
# For example, if 'unnecessary_column' is not needed, you can drop it
unnecessary_column = 'dispatching_base_num'
df_cleaned = df_cleaned.drop(columns=[unnecessary_column])

# Apply appropriate data types to columns
# For example, convert a column to datetime if it contains date information
df_cleaned['request_datetime',] = pd.to_datetime(df_cleaned['request_datetime'])


# Write the cleaned data to the cleaned folder as a Parquet file
cleaned_file_name = 'cleaned' + file_name
df_cleaned.to_parquet('local_file_cleaned.parquet')

# Upload the cleaned file to the GCS cleaned folder
blob_cleaned = bucket.blob(my-bigdata-project-ns-fhvhv/Cleaned + cleaned_file_name)
blob_cleaned.upload_from_filename('local_file_cleaned.parquet')

print("Data cleaning and transformation completed.")

/tmp/ipykernel_32622/1145484515.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[numeric_columns] = df_cleaned[numeric_columns].fillna(df_cleaned[numeric_columns].mean())
/opt/conda/miniconda3/lib/python3.11/site-packages/pandas/io/parquet.py:189: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


AttributeError: 'str' object has no attribute 'blob'

In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean

# Initialize Spark session
spark = SparkSession.builder.appName("NYC Taxi Data Cleaning").getOrCreate()

# Define bucket and file paths
bucket = 'my-bigdata-project-ns-fhvhv'
file_list = ['fhvhv_tripdata_2020-01.parquet',
             'fhvhv_tripdata_2020-02.parquet',
             'fhvhv_tripdata_2020-03.parquet',
             'fhvhv_tripdata_2020-04.parquet',
             'fhvhv_tripdata_2020-05.parquet',
             'fhvhv_tripdata_2020-06.parquet',
             'fhvhv_tripdata_2020-07.parquet',
             'fhvhv_tripdata_2020-08.parquet',
             'fhvhv_tripdata_2020-09.parquet',
             'fhvhv_tripdata_2020-10.parquet',
             'fhvhv_tripdata_2020-11.parquet',
             'fhvhv_tripdata_2020-12.parquet',
             'fhvhv_tripdata_2021-01.parquet',
             'fhvhv_tripdata_2021-02.parquet',
             'fhvhv_tripdata_2021-03.parquet',
             'fhvhv_tripdata_2021-04.parquet',
             'fhvhv_tripdata_2021-05.parquet',
             'fhvhv_tripdata_2021-06.parquet',
             'fhvhv_tripdata_2021-07.parquet',
             'fhvhv_tripdata_2021-08.parquet',
             'fhvhv_tripdata_2021-09.parquet',
             'fhvhv_tripdata_2021-10.parquet',
             'fhvhv_tripdata_2021-11.parquet',
             'fhvhv_tripdata_2021-12.parquet']

base_directory = f"gs://my-bigdata-project-ns-fhvhv/Landing/"
cleaned_directory = f"gs://my-bigdata-project-ns-fhvhv/Cleaned/"

# Loop through each file in the list
for file_name in file_list:
    file_path = base_directory + file_name
    
    # Load the data from the Parquet file
    df = spark.read.parquet(file_path)
    
    # Clean the data
    df_cleaned = df.dropna()  # Remove records with missing data
    
    # Check if there are records in the DataFrame before filling missing values
    if df_cleaned.count() > 0:
        # Identify numeric columns and fill missing values with column means
        numeric_columns = [c for c, t in df_cleaned.dtypes if t in ('double', 'int')]
        column_means = df_cleaned.select(*(mean(col(c)).alias(c) for c in numeric_columns)).collect()[0].asDict()
        df_cleaned = df_cleaned.fillna(column_means)
    
        # Drop unnecessary column
        df_cleaned = df_cleaned.drop("dispatching_base_num")
    
        # Convert request_datetime to timestamp
        df_cleaned = df_cleaned.withColumn("request_datetime", col("request_datetime").cast("timestamp"))
    
        # Write the cleaned data to Parquet format
        cleaned_file_path = cleaned_directory + 'cleaned_' + file_name
        df_cleaned.write.mode("overwrite").parquet(cleaned_file_path)
    else:
        print(f"No records found in {file_name}. Skipping data cleaning for this file.")

print("Data cleaning and transformation completed.")
